In [2]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import json
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

In [3]:
project = "kfdrc-harmonization/sd-z6mwd3h0-alignment-wgs-normal"
task_id = "8a9b7be4-3be0-4149-8239-6738abe9f1d9"
out_file = open("/Users/brownm28/Documents/playground/cram2gvcf.tsv", "w")
# task_id = "3c20cc8e-18d7-43f2-bc2c-4a76d38a88f8"
task = api.tasks.get(task_id)
jobs = {}
temp = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED":
        check = job.name.split('_')
        cmd = job.command_line
        if job.command_line == None:
            # pdb.set_trace()
            cmd = "embedded script or task retry"
            sys.stderr.write("WARN: Job " + job.name + " had null cmd\n")
        if check[-1] == "s":
            key = "_".join(check[:-2])
            if key not in temp:
                jobs[job.start_time] = {}
                jobs[job.start_time][key] = cmd
                temp[key] = 1
            else:
                temp[key] += 1
        else:
            jobs[job.start_time] = {}
            jobs[job.start_time][job.name] = cmd
out_file.write("Step\tType\tNum scatter\tCommand\n")
for rtime in sorted(jobs.keys()):
    for key in jobs[rtime]:
        rtype = "run step"
        sct = "NA"
        if key in temp and temp[key] > 1:
            rtype = "scatter"
            sct = str(temp[key])
        cmds = jobs[rtime][key].split('\n')
        for cmd in cmds:
            out_file.write(key + "\t" + rtype + "\t" + sct + "\t" + cmd + "\n")
out_file.close()


WARN: Job picard_intervallisttools had null cmd
WARN: Job untar_reference had null cmd
WARN: Job bundle_secondaries had null cmd
WARN: Job index_dbsnp had null cmd


In [5]:
import sys

# max desired col width
max_w = 200
tsv_in = open("/Users/brownm28/Documents/playground/kf_annot_sub.tsv")
out_md = open("/Users/brownm28/Documents/playground/kf_annot_sub.md", "w")
data = []
max_len = []


for line in tsv_in:
    info = line.rstrip('\n').split('\t')
    data.append(info)
    if len(max_len) == 0:
        for item in info:
            max_len.append(len(item))
    else:
        for i in range(len(max_len)):
            if len(info[i]) > max_w:
                max_len[i] = max_w
            elif len(info[i]) > max_len[i]:
                max_len[i] = len(info[i])
# print header first
d_ct = []
for i in range(len(data[0])):
    d_ct.append(len(data[0][i]))
    out_md.write(" | " + data[0][i] + "".join([" "] * max_len[i]))
    d_ct[i] += max_len[i]
out_md.write(" |\n")
for i in range(len(data[0])):
    out_md.write(" | " + "".join(["-"] * d_ct[i]))
out_md.write(" |\n")
# pdb.set_trace()
for i in range(1, len(data), 1):
    for j in range(len(data[i])):
        d_ct = len(data[i][j]) + 2
        out_md.write(" | " + data[i][j] + "".join([" "] * max_len[j]))
        d_ct += max_len[j]
    out_md.write(" |\n")
out_md.close()